In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc
import matplotlib.pyplot as plt
import seaborn as sns
building = pd.read_csv('../input/ashrae-energy-prediction/building_metadata.csv')
weath_train = pd.read_csv('../input/ashrae-energy-prediction/weather_train.csv')
# weath_test = pd.read_csv('../input/ashrae-energy-prediction/weather_test.csv')
train = pd.read_csv('../input/ashrae-energy-prediction/train.csv')
# test = pd.read_csv('../input/ashrae-energy-prediction/test.csv')

## EDA
- 결측치 확인 및 채움

In [ ]:
weath_train.isnull().sum()
## 결측치 60%이상 - cloud_coverage 제외 (50%기준)

In [ ]:
def preproposal(weath_train):
    # 날짜별 분류 -> 날짜별 결측치에 mean값으로 채움
    weath_train["datetime"] = pd.to_datetime(weath_train["timestamp"])
    weath_train["day"] = weath_train["datetime"].dt.day
    weath_train["week"] = weath_train["datetime"].dt.week
    weath_train["month"] = weath_train["datetime"].dt.month
    weath_train = weath_train.set_index(['site_id','day','month'])  

    # air_temperature
    air_temperature_fill = pd.DataFrame(weath_train.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
    weath_train.update(air_temperature_fill,overwrite=False)  
    # cloud_coverage
    cloud_coverage_fill = weath_train.groupby(['site_id','day','month'])['cloud_coverage'].mean()
    cloud_coverage_fill = pd.DataFrame(cloud_coverage_fill.fillna(method='ffill'),columns=["cloud_coverage"])
    weath_train.update(cloud_coverage_fill,overwrite=False)
    #dew_temperature
    due_temperature_fill = pd.DataFrame(weath_train.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
    weath_train.update(due_temperature_fill,overwrite=False)
    #sea_level_pressure
    sea_level_fill = weath_train.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
    sea_level_fill = pd.DataFrame(sea_level_fill.fillna(method='ffill'),columns=['sea_level_pressure'])
    weath_train.update(sea_level_fill,overwrite=False)
    #wind_direction
    wind_direction_fill =  pd.DataFrame(weath_train.groupby(['site_id','day','month'])['wind_direction'].mean(),columns=['wind_direction'])
    weath_train.update(wind_direction_fill,overwrite=False)
    #wind_speed
    wind_speed_fill =  pd.DataFrame(weath_train.groupby(['site_id','day','month'])['wind_speed'].mean(),columns=['wind_speed'])
    weath_train.update(wind_speed_fill,overwrite=False)
    #precip_depth_1_hr
    precip_depth_fill = weath_train.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
    precip_depth_fill = pd.DataFrame(precip_depth_fill.fillna(method='ffill'),columns=['precip_depth_1_hr'])
    weath_train.update(precip_depth_fill,overwrite=False)

    weath_train = weath_train.reset_index()
    weath_train = weath_train.drop(['datetime','day','week','month'],axis=1)
    return weath_train

In [ ]:
def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
weath_train = preproposal(weath_train)

In [ ]:
weath_train.isnull().sum()

In [ ]:
train.isnull().sum()

In [ ]:
building.isnull().sum()

In [ ]:
## floor_count, square_feet와의 상관계수가 높으므로 square_feet만 사용.
corr = building.loc[:,['floor_count','square_feet']].corr()
print(corr)
gc.collect()


## 전처리
1. Categorical 수정
2. Data merge 
3. 시간순으로 정렬
4. 시간/week day 추가
5. square_feet log 변환
6. 사용하지 않는 feature drop


In [ ]:
## Categorical 수정
building['primary_use'] = pd.factorize(building['primary_use'])[0]

In [ ]:
building = reduce_mem_usage(building, use_float16=True)
gc.collect()
train = reduce_mem_usage(train, use_float16=True)
gc.collect()
weath_train = reduce_mem_usage(weath_train, use_float16=True)

In [ ]:
## data merge
## 1. traing + building
merge_df = pd.merge(train, building, on = 'building_id', how='left')
gc.collect()

In [ ]:
merge_df.isnull().sum()

In [ ]:
##2. trian,builing + weather
merge_df = pd.merge(merge_df, weath_train, on = ['site_id', 'timestamp'], how='left')
gc.collect()
merge_df

In [ ]:
# plt.figure(figsize=(12,6))
# sns.lineplot(data=merge_df.iloc[:2000], x="timestamp", y="air_temperature")
# plt.show()
x = merge_df.groupby('timestamp')['air_temperature'].sum()
x.plot()

gc.collect()

In [ ]:
merge_interp = merge_df.loc[:, ['air_temperature', 'dew_temperature','precip_depth_1_hr', 'sea_level_pressure', 'wind_direction','wind_speed']]

for i in merge_interp.columns:
    merge_df[i] = merge_df[i].interpolate(method = 'linear')
gc.collect()

In [ ]:
x = merge_df.groupby('timestamp')['air_temperature'].sum()
x.plot()
gc.collect()

In [ ]:
## 전체 상관계수 확인.
corr = merge_df.drop(columns='meter_reading').corrwith(merge_df['meter_reading'])
corr.sort_values()

In [ ]:
def feature_eng(merge_df) :
    #전처리 시간 순으로 정렬
    merge_df.sort_values("timestamp")
    merge_df.reset_index(drop=True)

    # Add features
    merge_df["timestamp"] = pd.to_datetime(merge_df["timestamp"],format="%Y-%m-%d %H:%M:%S")
    merge_df["weekday"] = merge_df["timestamp"].dt.weekday
    merge_df["hour"] = merge_df["timestamp"].dt.hour
    merge_df["month"] = merge_df["timestamp"].dt.month
    merge_df['square_feet'] =  np.log1p(merge_df['square_feet'])
    merge_df['meter_reading'] =  np.log1p(merge_df['meter_reading'])
    ## 주의 할점 min/max -- log변환 차이(분포가 바뀜)

    drop = ['timestamp', 'year_built', 'floor_count', 'cloud_coverage']
    merge_df = merge_df.drop(drop, axis=1)
    gc.collect()
    return merge_df

def feature_eng_test(merge_df) :
    #전처리 시간 순으로 정렬
    merge_df.sort_values("timestamp")
    merge_df.reset_index(drop=True)

    # Add features
    merge_df["timestamp"] = pd.to_datetime(merge_df["timestamp"],format="%Y-%m-%d %H:%M:%S")
    merge_df["weekday"] = merge_df["timestamp"].dt.weekday
    merge_df["hour"] = merge_df["timestamp"].dt.hour
    merge_df["month"] = merge_df["timestamp"].dt.month
    merge_df['square_feet'] =  np.log1p(merge_df['square_feet'])
#     merge_df['meter_reading'] =  np.log1p(merge_df['meter_reading'])
    ## 주의 할점 min/max -- log변환 차이(분포가 바뀜)

    drop = ['timestamp', 'year_built', 'floor_count', 'cloud_coverage']
    merge_df = merge_df.drop(drop, axis=1)
    gc.collect()
    return merge_df

In [ ]:
merge_df= feature_eng(merge_df)
gc.collect()

In [ ]:
corr = merge_df.drop(columns='meter_reading').corrwith(merge_df['meter_reading'])
corr.sort_values()

In [ ]:
merge_df.info()

In [ ]:
merge_df.isnull().sum()
# optuna는 점심시간에
# lgbm gpu로 도는지 체크

In [ ]:
merge_df = reduce_mem_usage(merge_df, use_float16=True)

## training

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMRegressor

param_grid = {
    "max_bin" : 20,
    "learning_rate" : 0.01,
    "objective" : "regression",
    "boosting_type" : "gbdt",
    "metric" : "mae",
    "sub_feature" : 0.345,
    "bagging_fraction" : 0.85,
    "bagging_freq" : 40,
    "num_leaves" : 512,
    "min_data" : 500,
    "min_hessian" : 0.05,
    "verbose" : 2,
    "feature_fraction_seed" : 2,
    "bagging_seed" : 3,
    'device' : 'gpu'
}

model = LGBMRegressor(**param_grid)
gc.collect()

In [ ]:
# 첫번째 테스트용으로 사용하고, 실제 학습시에는 K-Fold CV를 사용합니다.
from sklearn.model_selection import train_test_split

X = merge_df.drop(columns=['meter_reading'])  # 학습에 사용하지 않을 column들을 drop.
y = merge_df.meter_reading

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0xC0FFEE)

In [ ]:
print("\nFitting LightGBM...")
model.fit(X_train, y_train)
gc.collect()

## prediction

In [ ]:
# metric은 그때마다 맞게 바꿔줘야 합니다.
##RMSLE 설정
from sklearn.metrics import mean_squared_log_error

evaluation_metric = mean_squared_log_error


In [ ]:
def rmsle(y,y1):
    return np.sqrt(np.mean(np.square(np.log1p(y)-np.log1p(y1))))

In [ ]:
print("Prediction")
pred_train = model.predict(X_train)
gc.collect()
pred_test = model.predict(X_test)
gc.collect()

In [ ]:
train_score =rmsle(y_train, pred_train)
test_score = rmsle(y_test, pred_test)
print("Train Score : %.4f" % train_score)
print("Test Score : %.4f" % test_score)

In [ ]:
pred_train
# np.exp(pred_train)-1

## hyper-parameter tuning

In [ ]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     "max_depth" : [8, 16, None],
#     "n_estimators" : [100, 300, 500],
#     "max_bin" : [20],
#     "learning_rate" : [0.001, 0.0025, 0.003],
#     "objective" : ["regression"],
#     "boosting_type" : ["gbdt"],
#     "metric" : ["mae"],
#     "sub_feature" : [0.345],
#     "bagging_fraction" : [0.7, 0.75, 0.85],
#     "bagging_freq" : [40],
#     "num_leaves" : [256, 512],
#     "min_data" : [500],
#     "verbose" : [-1], # 필수
#     "min_hessian" : [0.05],
#     "feature_fraction_seed" : [2],
#     "bagging_seed" : [3]
# }  # 3 x 3 x 3 x 3 x 2 = 162 ---> GridSearch가 탐색하는 Hyper-parameter 조합 수.

# # CV(Cross-Validation) : KFold(K등분)
# gcv = GridSearchCV(estimator=model, param_grid=param_grid, cv=5,
#                   n_jobs=-1, verbose=1)

# gcv.fit(X_train, y_train)  # 5 x 162 = 810.
# print("Best Estimator : ", gcv.best_estimator_) 
# gc.collect()

In [ ]:
# print("Prediction with Best Estimator")
# gcv_pred_train = gcv.predict(X_train)
# gc.collect()
# gcv_pred_test = gcv.predict(X_test)
# gc.collect()

# gcv_train_score = rmsle(y_train, gcv_pred_train)
# gcv_test_score = rmsle(y_test, gcv_pred_test)

# print("Train R2 Score : %.4f" % gcv_train_score)
# print("Test R2 Score : %.4f" % gcv_test_score)

In [ ]:
# print("Performance Gain") # 이걸로 성능 향상 확인.
# print("in train : ", -(train_score - gcv_train_score))
# print("in test : ", -(test_score - gcv_test_score))

In [ ]:
def optimizer(trial, X, y, K):
    # 조절할 hyper-parameter 조합을 적어줍니다.
#     n_estimators = trial.suggest_int('n_estimators', 100, 200)
#     max_depth = trial.suggest_int('max_depth', 4, 8)
#     max_features = trial.suggest_categorical('max_features', ['sqrt', 'auto', 'log2'])
    
    
#     # 원하는 모델을 지정합니다, optuna는 시간이 오래걸리기 때문에 저는 보통 RF로 일단 테스트를 해본 뒤에 LGBM을 사용합니다.
#     model = RandomForestRegressor(n_estimators=n_estimators,
#                                  max_depth=max_depth,
#                                  max_features=max_features)
    
    max_depth = trial.suggest_int('max_depth', 5, 20)
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_bin = trial.suggest_int('max_bin', 20,100)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.01)
 
    model = LGBMRegressor(max_depth=max_depth,
                          n_estimators=n_estimators,
                          max_bin=max_bin,
                          learning_rate=learning_rate,
                         device='gpu')

    # K-Fold Cross validation을 구현합니다.
    folds = KFold(n_splits=K)
    losses = []
    
    for train_idx, val_idx in folds.split(X, y):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]
        
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        loss = rmsle(y_val, preds)
        losses.append(loss)
    
    
    # K-Fold의 평균 loss값을 돌려줍니다.
    return np.mean(losses)

In [ ]:
from sklearn.model_selection import KFold
from functools import partial
import optuna
K = 5 # Kfold 수
opt_func = partial(optimizer, X=X_train, y=y_train, K=K)

study = optuna.create_study(study_name = 'LGMB',direction="minimize") # 최소/최대 어느 방향의 최적값을 구할 건지. # R2 score는 커질수록 좋음.
study.optimize(opt_func, n_trials=10)

In [ ]:
gc.collect()

In [ ]:
# optuna가 시도했던 모든 실험 관련 데이터
study.trials_dataframe()

In [ ]:
print("Best Score: %.4f" % study.best_value) # best score 출력
print("Best params: ", study.best_trial.params) # best score일 때의 하이퍼파라미터들

## 제출

In [ ]:
## test 전처리
weath_test = pd.read_csv('../input/ashrae-energy-prediction/weather_test.csv')
test = pd.read_csv('../input/ashrae-energy-prediction/test.csv')
weath_test = preproposal(weath_test)
gc.collect()
weath_test = reduce_mem_usage(weath_test, use_float16=True)
gc.collect()
test = reduce_mem_usage(test, use_float16=True)


In [ ]:
gc.collect()
merge_df = pd.merge(test, building, on = 'building_id', how='left')
##2. trian,builing + weather
merge_df = pd.merge(merge_df, weath_test, on = ['site_id', 'timestamp'], how='left')
gc.collect()

In [ ]:
merge_interp = merge_df.loc[:, ['air_temperature', 'dew_temperature','precip_depth_1_hr', 'sea_level_pressure', 'wind_direction','wind_speed']]

for i in merge_interp.columns:
    merge_df[i] = merge_df[i].interpolate(method = 'linear')
gc.collect()

merge_df= feature_eng_test(merge_df)
gc.collect()
merge_df = reduce_mem_usage(merge_df, use_float16=True)
gc.collect()

In [ ]:
X_test = merge_df.drop(columns='meter_reading')
gc.collect()

In [ ]:
model = LGBMRegressor(max_depth=study.best_trial.params['max_depth'],
                      n_estimators=study.best_trial.params['n_estimators'],
                      max_bin=study.best_trial.params['max_bin'],
                      learning_rate=study.best_trial.params['learning_rate'])

model.fit(X, y) # 전체 데이터로 학습
preds = model.predict(X_test)
preds